<center><h1>Management and Analysis of Physics Dataset (MOD. B) </h1></center>
<center><h2> Project 5 - Streaming processing of cosmic rays using Drift Tubes detectors</h2></center>
<center><h2>Group 2305</h2></center>

<center><style>
    table {font-size: 24px;}
</style></center>

| Last Name        | First Name         |Student ID|
|:----------------:|:------------------:|:--------------:|
| Bertinelli       | Gabriele           |1219907 (tri)   |
| Bhatti           | Roben              |2091187         |
| Bonato           | Diego              |2091250         |
| Cacciola         | Martina            |2097476         |

<left><h2> Part 2 - Data processing</h2></left>

### Import packages and modules

In [1]:
import os
import pandas as pd
import numpy as np
import json
import boto3
import json
import time

from tqdm        import tqdm

import kafka
from kafka       import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic



from pyspark.sql.functions import from_json, col, countDistinct, count, when, collect_list
from pyspark.sql import SparkSession

from pyspark.sql.types import StructField, StructType, StringType, DoubleType, IntegerType



### Session and Spark Context creation

With the following command we are asking to the master (and the resource manager) to create an application (Session) with the required resources and configurations.
In order to test the performance of the network, we varied the following parameters: 
- `spark.executor.instances`: controls the number of executors requested (Excecutors perform the actual computations on the data. They are responsible for executing these tasks in parallel and returning the results back to the driver program).
- `spark.executor.cores`: specifies the number of CPU cores that are allocated to each Executor
- `spark.sql.shuffle.partitions`: configures the number of partitions that are used when shuffling data for joins or aggregations.
- `spark.sql.execution.arrow.pyspark.enabled`: Apache Arrow is an in-memory columnar data format that is used in Spark to efficiently transfer data between JVM and Python processes. Arrow is available as an optimization when converting a Spark DataFrame to a Pandas DataFrame and when creating a Spark DataFrame from a Pandas DataFrame.

The Spark Context is the driver application program used to submit applications to Spark, and it is used to work with RDDs.

In [2]:
spark = SparkSession.builder \
    .master("##")\
    .appName("cosmic_rays_spark")\
    .config("spark.executor.instances", 10)\
    .config("spark.executor.cores",1)\
    .config("spark.sql.shuffle.partitions", 10)\
    .config("spark.executor.memory", "1500m")\
    .config("spark.driver.memory", "1g")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")\
    .config("spark.sql.adaptive.enabled", "false")\
    .config("spark.sql.adaptive.coalescePartitions.enabled", "false")\
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2")\
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")\
    .config('spark.eventLog.enabled', 'true')\
    .config('spark.sql.streaming.stateStore.stateSchemaCheck', 'false')\
    .config("spark.sql.streaming.numRecentProgressUpdates", 1000)\
    .getOrCreate()

:: loading settings :: url = jar:file:/home/bertinelli/spark/spark/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bertinelli/.ivy2/cache
The jars for the packages stored in: /home/bertinelli/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9f51bc1d-490e-477c-9f70-41543341391b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central

23/07/09 18:53:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
sc = spark.sparkContext
sc

### Producer creation
Here we create the producer, which is then used to send the message containing the “cleaned” data to the dashboard via producer.send (in the function) and foreachBatch (applying the function).

In [4]:
# define the list of brokers in the cluster
KAFKA_BOOTSTRAP_SERVER = '##' #kafka broker

In [5]:
# producer definition
producer = KafkaProducer(bootstrap_servers = KAFKA_BOOTSTRAP_SERVER,
                         batch_size=16000, #16MB
                          linger_ms=20  ) #ms

        
# KAFKA ADMIN is responsible for creating/deleting topics

# connecting to client 
kafka_admin = KafkaAdminClient(bootstrap_servers = KAFKA_BOOTSTRAP_SERVER)

In [6]:
kafka_admin.list_topics()

['data_clean', 'test_clean_1', 'data_raw', '__consumer_offsets']

### Data preprocessing

We create a DataFrame representing the stream of input lines from Kafka by connecting to the appropriate servers and topic.

In [7]:
# read streaming df from kafka
inputDF = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)\
    .option("kafkaConsumer.pollTimeoutMs", 1000)\
    .option('subscribe', 'data_raw')\
    .option("startingOffsets", "latest") \
    .load()

In [8]:
inputDF.isStreaming

True

We extract the values from the Kafka message, and we use `selectExpr` to create a table with the desired columns. 
`selectExpr()` is a function that takes a set of SQL expressions in a string to execute. This gives the ability to run SQL like expressions without creating a temporary table and views. `selectExpr()` just has one signature that takes SQL expression in a String and returns a new DataFrame.

In [9]:
# extract the value from the kafka message
rawraw_data = inputDF.select(col("value").cast("string")).alias("csv")

# split the csv line in the corresponding fields
raw_data = rawraw_data.selectExpr("cast(split(value, ',')[0] as int) as HEAD",
                                   "cast(split(value, ',')[1] as int) as FPGA",
                                   "cast(split(value, ',')[2] as int) as TDC_CHANNEL",
                                   "cast(split(value, ',')[3] as long) as ORBIT_CNT",
                                   "cast(split(value, ',')[4] as int) as BX_COUNTER",
                                   "cast(split(value, ',')[5] as double) as TDC_MEAS")

In [10]:
#raw_data.printSchema()

We select only the useful data (`HEAD=2`) and we apply a mapping between the data-format and the detectors, creating a new column `CHAMBER`. We eventually remove all rows with null values.

In [11]:
# selecting the wanted column
raw_data = raw_data.filter(raw_data.HEAD == 2)

# add CHAMBER column
raw_data = raw_data.withColumn("CHAMBER", \
                    when((raw_data.FPGA == 0) & (raw_data.TDC_CHANNEL>=0) & (raw_data.TDC_CHANNEL<64), 0) \
                   .when((raw_data.FPGA == 0) & (raw_data.TDC_CHANNEL>=64) & (raw_data.TDC_CHANNEL<128), 1 ) \
                   .when((raw_data.FPGA == 1) & (raw_data.TDC_CHANNEL>=0) & (raw_data.TDC_CHANNEL<64), 2 ) \
                   .when((raw_data.FPGA == 1) & (raw_data.TDC_CHANNEL>=64) & (raw_data.TDC_CHANNEL<128), 3 )\
                  ).na.drop()



### Data processing

We create a function that will be applied to each batch of data in order to compute the following piece of data:
1. total number of processed hits, post-clensing (1 value per batch) &rarr; `hit_count`
2. total number of processed hits, post-clensing, per chamber (4 values per batch) &rarr; `hit_count_chamber`
3. histogram of the counts of active `TDC_CHANNEL`, per chamber (4 arrays per batch) &rarr; `ch*_tdc_counts_list`
4. histogram of the total number of active `TDC_CHANNEL` in each `ORBIT_CNT`, per chamber (4 arrays per batch) &rarr; `ch*_orbit_counts_list`

Finally, we create a json message that will be sent to a Kafka topic via `producer.send()`.
Additionally, we print the time taken by each batch from the beginning of the data processing phase until the message is sent to the topic.

We use the `.persist()` function on the initial and intermediate dataframes for tasks 3 and 4.
It is a good practice to persist the DataFrame in memory before performing operations on it repeatedly. This helps in caching the DataFrame's partitions in memory and avoids unnecessary re-computation when the DataFrame is accessed multiple times.

In [12]:
from numpyencoder import NumpyEncoder

ID = -1

# function to apply to each batch: writes and sends a kafka message at the end
def batch_processing(df, epoch_id):

    df = df.persist()

    # 1: total number of processed hits, post-cleansing (1 value per batch)
    
    start = time.time()
    
    hit_count = df.count()
    
    
    # 2: total number of processed hits, post-cleansing, per chamber (4 values per batch)


    hit_count_chamber = df.groupby('CHAMBER').agg(count('TDC_CHANNEL').alias('HIT_COUNT'))\
                        .sort("CHAMBER").select('HIT_COUNT')\
                        .agg(collect_list('HIT_COUNT')).collect()



    # 3: histogram of the counts of active TDC_CHANNEL, per chamber (4 arrays per batch)

    tdc_counts = df.groupby(['CHAMBER','TDC_CHANNEL']).agg(count('TDC_CHANNEL').alias('TDC_COUNTS'))
    tdc_counts = tdc_counts.persist()

    # Filter the tdc_counts DataFrame for each chamber 
    
    ch0_tdc_counts = tdc_counts.filter(tdc_counts.CHAMBER == 0).select('TDC_CHANNEL','TDC_COUNTS')\
                    .sort("TDC_CHANNEL").toPandas()
    
    ch1_tdc_counts = tdc_counts.filter(tdc_counts.CHAMBER == 1).select('TDC_CHANNEL','TDC_COUNTS')\
                    .sort("TDC_CHANNEL").toPandas()
    
    ch2_tdc_counts = tdc_counts.filter(tdc_counts.CHAMBER == 2).select('TDC_CHANNEL','TDC_COUNTS')\
                    .sort("TDC_CHANNEL").toPandas()
    
    ch3_tdc_counts = tdc_counts.filter(tdc_counts.CHAMBER == 3).select('TDC_CHANNEL','TDC_COUNTS')\
                    .sort("TDC_CHANNEL").toPandas()

    
    
    #Save it in a list
    
    ch0_tdc_channels_list = list(ch0_tdc_counts['TDC_CHANNEL'])
    ch0_tdc_counts_list   = list(ch0_tdc_counts['TDC_COUNTS'])

    ch1_tdc_channels_list = list(ch1_tdc_counts['TDC_CHANNEL'])
    ch1_tdc_counts_list   = list(ch1_tdc_counts['TDC_COUNTS'])
    
    ch2_tdc_channels_list = list(ch2_tdc_counts['TDC_CHANNEL'])
    ch2_tdc_counts_list   = list(ch2_tdc_counts['TDC_COUNTS'])
    
    ch3_tdc_channels_list = list(ch3_tdc_counts['TDC_CHANNEL'])
    ch3_tdc_counts_list   = list(ch3_tdc_counts['TDC_COUNTS'])
    
    

    # 4: histogram of the total number of active TDC_CHANNEL in each ORBIT_CNT, per chamber (4 arrays per batch)

    orbit_count=df.groupby(['CHAMBER','ORBIT_CNT']).agg(countDistinct("TDC_CHANNEL").alias('TDC_ORBIT'))
    orbit_count = orbit_count.persist()

    ch0_orbit_counts = orbit_count.filter(orbit_count.CHAMBER == 0).select('ORBIT_CNT','TDC_ORBIT')\
                    .sort("ORBIT_CNT").toPandas()
    
    ch1_orbit_counts = orbit_count.filter(orbit_count.CHAMBER == 1).select('ORBIT_CNT','TDC_ORBIT')\
                    .sort("ORBIT_CNT").toPandas()
    
    ch2_orbit_counts = orbit_count.filter(orbit_count.CHAMBER == 2).select('ORBIT_CNT','TDC_ORBIT')\
                    .sort("ORBIT_CNT").toPandas()
    
    ch3_orbit_counts = orbit_count.filter(orbit_count.CHAMBER == 3).select('ORBIT_CNT','TDC_ORBIT')\
                    .sort("ORBIT_CNT").toPandas()
    
    #Save it in a list
    
    ch0_orbit_list          = list(ch0_orbit_counts['ORBIT_CNT'])
    ch0_orbit_counts_list   = list(ch0_orbit_counts['TDC_ORBIT'])

    ch1_orbit_list          = list(ch1_orbit_counts['ORBIT_CNT'])
    ch1_orbit_counts_list   = list(ch1_orbit_counts['TDC_ORBIT'])

    ch2_orbit_list          = list(ch2_orbit_counts['ORBIT_CNT'])
    ch2_orbit_counts_list   = list(ch2_orbit_counts['TDC_ORBIT'])
    
    ch3_orbit_list          = list(ch3_orbit_counts['ORBIT_CNT'])
    ch3_orbit_counts_list   = list(ch3_orbit_counts['TDC_ORBIT'])
    

 
    
    df.unpersist()
    tdc_counts.unpersist()
    orbit_count.unpersist()
        
    
    global ID
    ID += 1

    # prepare message to send to kafka
    
    msg = {

        'msg_ID': ID,
        'hit_count': hit_count,
        'hit_count_chamber': hit_count_chamber[0][0],
           
        'tdc_counts_chamber': {
            '0': {
                'bin_edges': ch0_tdc_channels_list,
                'hist_counts': ch0_tdc_counts_list
            },
            '1': {
                'bin_edges': ch1_tdc_channels_list,
                'hist_counts': ch1_tdc_counts_list
            },
            '2': {
                'bin_edges': ch2_tdc_channels_list,
                'hist_counts': ch2_tdc_counts_list
            },
            '3': {
                'bin_edges': ch3_tdc_channels_list,
                'hist_counts': ch3_tdc_counts_list
            }
        },
        'active_tdc_chamber': {
            '0': {
                'bin_edges': ch0_orbit_list,
                'hist_counts': ch0_orbit_counts_list
            },
            '1': {
                'bin_edges': ch1_orbit_list,
                'hist_counts': ch1_orbit_counts_list
            },
            '2': {
                'bin_edges': ch2_orbit_list,
                'hist_counts': ch2_orbit_counts_list
            },
            '3': {
                'bin_edges': ch3_orbit_list,
                'hist_counts': ch3_orbit_counts_list
            }
        }
    }
   

    
    
    
    producer.send('data_clean', json.dumps(msg).encode('utf-8'))
    producer.flush()
    #producer.poll(0)
    
    stop = time.time() - start
    
    print(f'\nTime: {stop}')

### Sending message to Kafka topic

Here, we apply the function defined earlier, `batch_processing()`, to each batch in the streaming dataset `raw_data`, through the `foreachBatch()` function.
`.trigger()` is a method that allows you to define the trigger for a streaming query. A trigger determines when the streaming query should be executed and how often it should process the data. To investigate performance differences, we varies the `processingTime` parameter.

In [13]:
query = raw_data.writeStream\
            .outputMode("update")\
            .foreachBatch(batch_processing)\
            .option("checkpointLocation", "checkpoint")\
            .trigger(processingTime='5 seconds')\
            .start()
query.awaitTermination(5)

False


Time: 23.333239793777466
23/07/09 18:54:13 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 26411 milliseconds


23/07/09 18:54:16 WARN TaskSetManager: Lost task 9.0 in stage 56.0 (TID 210) (10.67.22.77 executor 0): java.util.concurrent.TimeoutException: Cannot fetch record for offset 3576 in 1000 milliseconds
	at org.apache.spark.sql.kafka010.consumer.InternalKafkaConsumer.fetch(KafkaDataConsumer.scala:97)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchData(KafkaDataConsumer.scala:552)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchRecord(KafkaDataConsumer.scala:476)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.$anonfun$get$1(KafkaDataConsumer.scala:313)
	at org.apache.spark.util.UninterruptibleThread.runUninterruptibly(UninterruptibleThread.scala:77)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.runUninterruptiblyIfPossible(KafkaDataConsumer.scala:618)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.get(KafkaDataConsumer.scala:290)
	at org.apache.spark.sql.kafka010.KafkaBatchPartitionReader.next(KafkaBatchPartitio

23/07/09 18:54:17 WARN TaskSetManager: Lost task 2.0 in stage 56.0 (TID 203) (10.67.22.111 executor 6): java.util.concurrent.TimeoutException: Cannot fetch record for offset 3468 in 1000 milliseconds
	at org.apache.spark.sql.kafka010.consumer.InternalKafkaConsumer.fetch(KafkaDataConsumer.scala:97)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchData(KafkaDataConsumer.scala:552)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchRecord(KafkaDataConsumer.scala:476)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.$anonfun$get$1(KafkaDataConsumer.scala:313)
	at org.apache.spark.util.UninterruptibleThread.runUninterruptibly(UninterruptibleThread.scala:77)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.runUninterruptiblyIfPossible(KafkaDataConsumer.scala:618)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.get(KafkaDataConsumer.scala:290)
	at org.apache.spark.sql.kafka010.KafkaBatchPartitionReader.next(KafkaBatchPartiti


Time: 17.259406089782715
23/07/09 18:54:30 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 17611 milliseconds


...


Time: 2.457263231277466

Time: 2.333871841430664



Time: 2.2513229846954346

Time: 1.9208505153656006



Time: 2.380497694015503



Time: 2.764342784881592



Time: 2.415839672088623


### Metrics

To analyze the performance of the various combinations of metrics, we save the metrics in a .json file so they can be analyzed and compared later.

In [ ]:
a = query.recentProgress

#kp = kafkapartitions
#em exectutor memory
#dm driver memory
#sp shuffle partitions
#aT arrow true
#w workers
with open('metriche/10ex_1core_6sp_aT_2w_6kp_5000batch_1secProcTime.json', 'w') as file:
    # Perform file operations
    json.dump(a, file)

In [14]:
print("Print this to stop the code and avoid to stop spark accidentally")

Print this to stop the code and avoid to stop spark accidentally


### Stop workers and master

In [15]:
# stop the running Spark context and Spark session
sc.stop()

spark.stop()